In [88]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from ipywidgets import widgets
from IPython.display import display
from IPython.display import clear_output

user_box = widgets.Text("userid")
display(user_box)
location_box = widgets.Text("Location")
display(location_box)

userId = 0
locationName = ""

def handle_user_submit(sender):
    global userId
    userId = user_box.value
    clear_output()
    show_distinct_locations_for_user(userId)
    
user_box.on_submit(handle_user_submit)

def show_distinct_locations_for_user(userId):
    distinct_locations_user = sqlContext.sql(
                                    "select distinct l.name "
                                    + "from locations l "
                                    + "join outputs o on l.id = o.locationId "
                                    + "join types t on t.id = o.typeId "
                                    + "where l.userId = '" + str(userId) + "' "
                                    + "and t.name = 'THERMO'"
                                    + "order by l.name")
    for element in distinct_locations_user.collect():
        print element.name

def handle_location_submit(sender):
    global locationName
    locationName = location_box.value
    clear_output()
    print "Computing plot"
    compute_plot()
    
location_box.on_submit(handle_location_submit)

def compute_plot():
    set_temps = sqlContext.sql("select ol.time, ol.value, l.userid, o.id, l.name "
                          + "from OutputLogs ol "
                          + "join Outputs o on ol.OutputID = o.id "
                          + "join Locations l on o.locationid = l.id "
                          + "join Types t on o.typeid = t.id "
                          + "where l.Userid = '" + str(userId) + "' and t.Name = 'THERMO' "
                          + "and l.name = '" + locationName + "' and ol.time > '2016-02-22' "
                          + "order by ol.time")

    measured_temps = sqlContext.sql("select oghd.time, oghd.value, l.userid, o.id, l.name "
                          + "from OutputGraphHourData oghd "
                          + "join Outputs o on oghd.outputid = o.id "
                          + "join Locations l on o.locationid = l.id "
                          + "join Types t on o.typeid = t.id "
                          + "where l.Userid = '" + str(userId) + "' "
                          + "and l.name = '" + locationName + "' and oghd.time > '2016-02-22' "
                          + "order by oghd.time")
    
    fig, ax = plt.subplots(figsize=(20,10))
    plt.plot(set_temps.select('time').collect(), 
             set_temps.select('value').collect(),
             label = "set")
    plt.plot(measured_temps.select('time').collect(), 
             measured_temps.select('value').collect(),
            label = "measured")
    
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y, %-Hh'))
    ax.xaxis.set_minor_locator(mdates.HourLocator())
    fig.autofmt_xdate()
    
    plt.title("Temperatures of " + locationName + " for user " + str(userId))
    plt.legend()
    plt.grid()
    
    clear_output() 
    
    plt.show()

Computing plot


In [ ]:
## For performance gain set_temps and measured_temps in hive table